In [19]:
!pip install llama-index-core llama-index-llms-openai python-dotenv llama-index-tools-yahoo-finance --quiet


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from dotenv import load_dotenv
# so everything with LlamaIndex works correctly in a notebook
import nest_asyncio

nest_asyncio.apply()
load_dotenv()

True

In [32]:
##### CREATE TOOLS #####
from llama_index.tools.yahoo_finance import YahooFinanceToolSpec

def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b


def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

async def set_name(ctx: Context, name: str) -> str:
    state = await ctx.get("state")
    state["name"] = name
    await ctx.set("state", state)
    return f"Name set to {name}"

finance_tools = YahooFinanceToolSpec().to_tool_list()
finance_tools.extend([multiply, add, set_name])

In [33]:
##### INITIALISE LLM #####
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o-mini")

In [34]:
##### INITIALISE AGENT #####
from llama_index.core.agent.workflow import AgentWorkflow

workflow = AgentWorkflow.from_tools_or_functions(
    finance_tools,
    llm=llm,
    system_prompt="You are an agent that can perform basic mathematical operations using tools.",
    initial_state={"name": "unset"}
)

In [35]:
##### CREATE STATE TO MAINTAIN #####
from llama_index.core.workflow import Context

ctx = Context(workflow)

In [36]:
##### ASK A QUESTION #####

In [37]:
response = await workflow.run(user_msg="What is 20+(2*4)?", ctx=ctx)
print(response)

The result of the calculation \(20 + (2 \times 4)\) is 28.


In [38]:
response = await workflow.run(user_msg="What's the current stock price of NVIDIA?", ctx=ctx)
print(response)

The current stock price of NVIDIA (NVDA) is $117.70.


In [39]:
response = await workflow.run(user_msg="Hi, my name is Jean Luc Piccard", ctx=ctx)
print(response)

Nice to meet you, Jean Luc Piccard!


In [40]:
response = await workflow.run(user_msg="What's my name?", ctx=ctx)
print(response)

Your name is Jean Luc Piccard.


In [42]:
##### SAVE AND RETRIEVE STATE #####
from llama_index.core.workflow import JsonPickleSerializer, JsonSerializer

ctx_dict = ctx.to_dict(serializer=JsonSerializer())

restored_ctx = Context.from_dict(
    workflow, ctx_dict, serializer=JsonSerializer()
)

response = await workflow.run(user_msg="What's my name?", ctx=ctx)
print(response)

Your name is Jean Luc Piccard.


In [43]:
##### TOOLS WITH STATE #####

state = await ctx.get("state")
print("Name as stored in state: ", state["name"])

Name as stored in state:  Jean Luc Piccard
